In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.metrics import classification_report

#Resnet-50 model was created in majority by Angel Picon with specific help from Leonel Torres on the loading cov images with label as he figured this section out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# google drive mount
from google.colab import drive
drive.mount('/content/drive')

# Dataset Class
class COVIDDataset(Dataset):
    def __init__(self, transform=None):
        self.samples = []
        self.transform = transform

        # Directory for images
        covid_folder = "/content/drive/MyDrive/Cov19 Dataset/Early Cov"
        non_covid_folder = "/content/drive/MyDrive/Cov19 Dataset/Control"

        # Load COV images Label 1
        for img in os.listdir(covid_folder):
            if img.lower().endswith(('.png', '.jpg', '.jpeg')):
                self.samples.append((os.path.join(covid_folder, img), 1))

        # Load NON-COV images label 0
        for img in os.listdir(non_covid_folder):
            if img.lower().endswith(('.png', '.jpg', '.jpeg')):
                self.samples.append((os.path.join(non_covid_folder, img), 0))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label

# Transform to 224
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Resnet model
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 2)  # Binary classification
model = model.to(device)

# Training the model
def train(dataloader, epochs=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    model.train()
    for epoch in range(epochs):
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs} complete")

# Model Evaluation
def evaluate(dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds,
                              target_names=["Non-COVID", "COVID"]))

# 8. Main Execution
if __name__ == "main":
    # Create dataset
    dataset = COVIDDataset(transform=transform)

    # 80 20 dataset split
    train_size = int(0.8 * len(dataset))
    train_set, test_set = torch.utils.data.random_split(dataset, [train_size, len(dataset)-train_size])

    # Create dataloaders
    train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=16)

    # Train and evaluate
    train(train_loader)
    evaluate(test_loader)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/3 complete
Epoch 2/3 complete
Epoch 3/3 complete

Classification Report:
              precision    recall  f1-score   support

   Non-COVID       1.00      0.81      0.90       106
       COVID       0.82      1.00      0.90        94

    accuracy                           0.90       200
   macro avg       0.91      0.91      0.90       200
weighted avg       0.92      0.90      0.90       200



In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

#main implementation remained the same so same code written by Angel Picon. Leonel introduced the idea of using Random Forrest after our presentation so the random forrest implementation was written by him.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


from google.colab import drive
drive.mount('/content/drive')

# Same implementation as previous
class COVIDDataset(Dataset):
    def __init__(self, transform=None):
        self.samples = []
        self.transform = transform

        covid_folder = "/content/drive/MyDrive/Cov19 Dataset/Early Cov"
        non_covid_folder = "/content/drive/MyDrive/Cov19 Dataset/Control"

        for img in os.listdir(covid_folder):
            if img.lower().endswith(('.png', '.jpg', '.jpeg')):
                self.samples.append((os.path.join(covid_folder, img), 1))

        for img in os.listdir(non_covid_folder):
            if img.lower().endswith(('.png', '.jpg', '.jpeg')):
                self.samples.append((os.path.join(non_covid_folder, img), 0))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 80-20 data split
dataset = COVIDDataset(transform=transform)
train_size = int(0.8 * len(dataset))
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

# Declaring feature extractions
feature_extractor = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-1])
feature_extractor = feature_extractor.to(device)
feature_extractor.eval()

# Function for creating features including flatten which is neccesary in Random forrest
def extract_features(dataset):
    loader = DataLoader(dataset, batch_size=16, shuffle=False)
    features = []
    labels = []

    with torch.no_grad():
        for imgs, lbls in tqdm(loader, desc="Extracting Features"):
            imgs = imgs.to(device)
            out = feature_extractor(imgs)
            out = out.view(out.size(0), -1)  #
            features.append(out.cpu().numpy())
            labels.extend(lbls)

    return np.concatenate(features), np.array(labels)

# Extract features to train sets
X_train_feat, y_train = extract_features(train_set)
X_test_feat, y_test = extract_features(test_set)

# Training random forrest as stolen from previous lectures
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_feat, y_train)

# Evaluation report
y_pred = rf_model.predict(X_test_feat)
print("\n=== Random Forest Classification Report ===")
print(classification_report(y_test, y_pred, target_names=["Non-COVID", "COVID"]))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 177MB/s]
Extracting Features: 100%|██████████| 13/13 [00:41<00:00,  3.17s/it]



=== Random Forest Classification Report ===
              precision    recall  f1-score   support

   Non-COVID       0.78      0.77      0.78        99
       COVID       0.78      0.79      0.78       101

    accuracy                           0.78       200
   macro avg       0.78      0.78      0.78       200
weighted avg       0.78      0.78      0.78       200



In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import random
import numpy as np
import cv2
from google.colab import drive
from datetime import datetime
from sklearn.metrics import classification_report

#The base code all the way up to the ModelEVAL function was created by Angel Picon. Everything past ModelEval was written by Leonel Torres

        # Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Mount drive
drive.mount('/content/drive')

class XRayIMGDataset(Dataset):
    def __init__(self):
        self.image_paths = []
        self.labels = []

        # Direct paths to our dataset folders
        covid_dir = "/content/drive/MyDrive/Cov19 Dataset/Early Cov"
        normal_dir = "/content/drive/MyDrive/Cov19 Dataset/Control"

        # Find Positive covid-19 xray images ending with png, jpg, jpeg
        cov_files = [f for f in os.listdir(covid_dir) if f.lower().endswith(('png','jpg','jpeg'))]
        random.shuffle(cov_files)
        for img in cov_files[:500]:  # Temp limit of files for max number of files in folder
            self.image_paths.append(os.path.join(covid_dir, img))
            self.labels.append(1)  # Label is 1 for positive cases

        # Find negative covid-19 xray images ending with png, jpg, jpeg
        print(f"{datetime.now().strftime('%H:%M')} - Loading normal images...")
        norm_files = [f for f in os.listdir(normal_dir) if f.lower().endswith(('png','jpg','jpeg'))]
        random.shuffle(norm_files)
        for img in norm_files[:500]:   # Temp limit of files for max number of files in folder
            self.image_paths.append(os.path.join(normal_dir, img))
            self.labels.append(0)  # Label is 2 for negative cases


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            # Convert to grayscale for Otsu thresholding
            img = Image.open(img_path).convert('L')
            img_np = np.array(img)

            # Apply Otsu thresholding
            thresh = cv2.adaptiveThreshold(
    img_np, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C, #guasian blur to higk
    cv2.THRESH_BINARY, 11, 2
)

            # Convert back to RGB for ResNet
            img_rgb = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
            img_pil = Image.fromarray(img_rgb)

            # Resize and convert to tensor
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor()
            ])
            img = transform(img_pil)

            return img, self.labels[idx]
        except Exception as e:
            return torch.zeros(3, 224, 224), 0  # Return blank image if error

# Use ResNet50 model
try:
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, 2)  # Binary implmentation
    )
    model = model.to(device)
except Exception as e:
    raise

# Training model using CrossEntropy and 3 epochs
def Model_TRAIN(data_loader, epochs=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        batch_count = 0


    for batch_idx, (images, labels) in enumerate(data_loader):
                if images is None:
                    continue

                images, labels = images.to(device), labels.to(device)

                #Training the model
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                batch_count += 1

                #check with model after each round to ensure model is learning (Decrease in average loss)
                if batch_idx % 30 == 0:
                    avg_loss = total_loss / batch_count
                    print(f"{datetime.now().strftime('%H:%M')} - Batch {batch_idx} | Loss: {avg_loss:.4f}")


def Model_EVAL(data_loader):
    model.eval()
    all_preds = []
    all_labels = []

    try:
        with torch.no_grad():
            for batch_idx, (images, labels) in enumerate(data_loader):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                if batch_idx % 20 == 0:
                    print(f"{datetime.now().strftime('%H:%M')} - Evaluated batch {batch_idx}")

        print("\nClassification Report:")
        print(classification_report(all_labels, all_preds, target_names=["Normal", "COVID"]))
    except Exception as e:
        print(f"{datetime.now().strftime('%H:%M')} - Evaluation failed: {str(e)}")
        raise

if __name__ == "__main__":

    try:
        # Load and prepare dataset
        dataset = XRayIMGDataset()

        # Split into training and test sets
        train_size = int(0.8 * len(dataset))
        test_size = len(dataset) - train_size
        train_set, test_set = torch.utils.data.random_split(
            dataset,
            [train_size, test_size]
        )

        # Create data loaders
        train_loader = DataLoader(train_set, batch_size=12, shuffle=True)
        test_loader = DataLoader(test_set, batch_size=12)

        # Train the model
        Model_TRAIN(train_loader, epochs=3)

        # Evaluate performance
        Model_EVAL(test_loader)

        #error handling in case cannot find dataset
    except Exception as e:
        print(f"\n{datetime.now().strftime('%H:%M')} - Critical error: {str(e)}")
    finally:
        print(f"\n{datetime.now().strftime('%H:%M')} - Execution complete")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
02:08 - Loading normal images...
02:08 - Batch 0 | Loss: 0.7242
02:08 - Batch 30 | Loss: 0.5681
02:08 - Batch 60 | Loss: 0.4741
02:08 - Evaluated batch 0

Classification Report:
              precision    recall  f1-score   support

      Normal       0.85      0.95      0.90        96
       COVID       0.95      0.85      0.89       104

    accuracy                           0.90       200
   macro avg       0.90      0.90      0.89       200
weighted avg       0.90      0.90      0.89       200


02:08 - Execution complete
